In [0]:
import numpy as np
import pandas as pd
import tqdm

import catboost as cat
from catboost import CatBoostClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lgb

In [0]:
%%time
edges = pd.read_csv('./edges.csv')
ids = pd.read_csv('./ids.csv')
vertices = pd.read_csv('./vertices.csv')

CPU times: user 3.38 s, sys: 329 ms, total: 3.71 s
Wall time: 3.71 s


In [0]:
vertices['main_okved'] = vertices['main_okved'].astype(str)

In [0]:
np.random.seed(7777)

In [0]:
result = pd.DataFrame(columns=['id_1', 'id_2'])

In [0]:
# для каждой вершины из ids с помощью catboost найдем 1000 самых вероятных ребер
for i in tqdm.tqdm(ids.id):
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    df1 = edges[edges['id_1'] == i].reset_index()
    df2 = edges[edges['id_2'] == i].reset_index()

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
    df['target'] = 1
    
    df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)
    
    
    X = df[['main_okved', 'region_code', 'company_type']]
    y = df['target']
    
    
    model = CatBoostClassifier(iterations=100, verbose=False)
    cat_features = [0,1,2] # все признаки категориальные
    
    model.fit(X, y, cat_features)

    preds = model.predict_proba(X)[:,1]

    df['preds'] = preds
    df['id_2'] = i
    
    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
    res.columns = ['id_1', 'id_2']
    
    result = result.append(res, ignore_index=True, sort=False)

 28%|██▊       | 28/100 [26:00<1:05:35, 54.66s/it]

Training has stopped (degenerate solution on iteration 93, probably too small l2-regularization, try to increase it)


100%|██████████| 100/100 [1:32:34<00:00, 54.28s/it]


In [0]:
X['n_common_neighbors'] = X.apply(lambda row:
            len(tuple(nx.common_neighbors(g, row['node1'], row['node2']))), axis=1)
pref_atts = nx.preferential_attachment(g, X[['node1', 'node2']].values)

X['pref_att'] = [pref_att for u_, v_, pref_att in pref_atts]
jacc_coefs = nx.jaccard_coefficient(g, X[['node1', 'node2']].values)

X['jacc_coef'] = [jacc for u_, v_, jacc in jacc_coefs]
adar_coefs = nx.adamic_adar_index(g, X[['node1', 'node2']].values)

X['adar_coef'] = [adar for u_, v_, adar in adar_coefs]
katz_dict = nx.katz_centrality(g, alpha=0.01)

X['katz_sum'] = X.apply(lambda row: katz_dict[row['node1']] +
                        katz_dict[row['node2']], axis=1)

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_val=sc.transform(x_val)

# Training
d_train = lgb.Dataset(x_train, label=y_train)
d_val = lgb.Dataset(x_val, label=y_val)
d_test=lgb.Dataset(x_test,label=y_test)

params = {
            'objective': 'binary',
            'boosting_type': 'gbdt',
#             'boosting_type': 'rf',
            'nthread': 4,
            'learning_rate': 0.02,  # 02,
            'num_leaves': 15,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.7,
            'bagging_freq': 1,
            'early_stopping_round':5,
            'max_depth':2,
            'reg_alpha': 0.041545473,
            'reg_lambda': 0.0735294,
            'min_split_gain': 0.0222415,
            'min_child_weight': 60,
            'seed': 0,
            'verbose': -1,
            'metric': 'auc',
}


clf = lgb.train(params, d_train, 20,d_val)

y_pred=clf.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)
print(accuracy)
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_pred))

In [0]:
testdata = pd.read_csv('test.csv')
x_testing = testdata.iloc[:,0:FEATURE_SIZE].values
x_testing = sc.transform(x_testing)
y_pred=clf.predict(x_testing)

In [0]:
import csv
with open("result.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Id","Prediction"])
    test_id=1
    for prediction in y_pred:
        writer.writerow([test_id,prediction])
        test_id+=1